In [ ]:
#default_exp EditOperations

In [ ]:
#export
#hide
from typing import List, Union

In [ ]:
#hide
import sys
sys.path.append("..")

from hephaestus.AbstractMethod import *

# EditOperations

> Defines classes that represent the following edit operations: InsertOperation, DeleteOperation, and ReplaceOperation

In [ ]:
#export
class InsertOperation:
    """
    Creates an `InsertOperation` which defines the insertion of a token into an `AbstractMethod`. Applying the
    operation will insert token `newToken` at index `tokenIndex`.
    """

    def __init__(self, tokenIndex: int, newToken: str) -> None:
        self.__tokenIndex = tokenIndex
        self.__newToken = newToken
    
    def __str__(self) -> str:
        return 'insert token "{}" at index {}'.format(self.__newToken, self.__tokenIndex)
    
    def __repr__(self) -> str:
        return str(self)
    
    def applyToTokens(self, tokens: List[str]) -> None:
        """
        Applies the insertion to the given list of tokens. Raises an error if the token index is out of bounds.
        """

        if self.__tokenIndex > len(tokens):
            raise IndexError("InsertOperation: cannot insert at index {} (out of bounds)".format(self.__tokenIndex))
        
        tokens.insert(self.__tokenIndex, self.__newToken)

In [ ]:
method = AbstractMethod("public void METHOD_1 ( ) { }")
method

'public void METHOD_1 ( ) { }'

In [ ]:
method.applyEditOperation(InsertOperation(1, "static"))
method

'public static void METHOD_1 ( ) { }'

In [ ]:
#export
class DeleteOperation:
    """
    Creates a `DeleteOperation` which defines the deletion of a token from an `AbstractMethod`. Applying the
    operation will delete the token at index `tokenIndex`.
    """

    def __init__(self, tokenIndex: int) -> None:
        self.__tokenIndex = tokenIndex
    
    def __str__(self) -> str:
        return "delete token at index {}".format(self.__tokenIndex)

    def __repr__(self) -> str:
        return str(self)
    
    def applyToTokens(self, tokens: List[str]) -> None:
        """
        Applies the deletion to the given list of tokens. Raises an error if the token index is out of bounds.
        """

        if self.__tokenIndex >= len(tokens):
            raise IndexError("DeleteOperation: cannot delete at index {} (out of bounds)".format(self.__tokenIndex))

        del tokens[self.__tokenIndex]

In [ ]:
method = AbstractMethod("public static void METHOD_1 ( ) { }")
method

'public static void METHOD_1 ( ) { }'

In [ ]:
method.applyEditOperation(DeleteOperation(1))
method

'public void METHOD_1 ( ) { }'

In [ ]:
#export
class ReplaceOperation:
    """
    Creates a `ReplaceOperation` which defines the replacement of a token in an `AbstractMethod`. Applying the
    operation will replace the token at index `tokenIndex` with token `newToken`.
    """

    def __init__(self, tokenIndex: int, newToken: str) -> None:
        self.__tokenIndex = tokenIndex
        self.__newToken = newToken
    
    def __str__(self) -> str:
        return 'replace token at index {} with "{}"'.format(self.__tokenIndex, self.__newToken)

    def __repr__(self) -> str:
        return str(self)
    
    def applyToTokens(self, tokens: List[str]) -> None:
        """
        Applies the replacement to the given list of tokens. Raises an error if the token index is out of bounds.
        """

        if self.__tokenIndex >= len(tokens):
            raise IndexError("ReplaceOperation: cannot replace at index {} (out of bounds)".format(self.__tokenIndex))

        tokens[self.__tokenIndex] = self.__newToken

In [ ]:
method = AbstractMethod("public void METHOD_1 ( ) { }")
method

'public void METHOD_1 ( ) { }'

In [ ]:
method.applyEditOperation(ReplaceOperation(0, "private"))
method

'private void METHOD_1 ( ) { }'

In [ ]:
#export
EditOperation = Union[InsertOperation, DeleteOperation, ReplaceOperation]